In [1]:
import configparser
import unittest
import datetime
import sys
import pandas as pd

# from tqdm import tqdm_notebook

sys.path.append("/home/sergey/drclinics/common")
from universal_connection import UniversalConnection, DBType

sys.path.append("/home/sergey/drclinics/reports")
# from report_utils import excel_report

import numpy as np
from log import log

In [2]:
from region import get_region_by_two_bases, clean_region

In [3]:
sql = """
with temp as (
select a.id appoint_id,
	   p.person_id,
	   per.phone,
	   --per.partner_id,
	   --par."name", 
	   case when pc.product_id in (
            select id from product
            where full_name like '%ВЭБ%') then 'VEBMED'
        	else 'ПРОЧИЕ'
        end "name",
	   DATE(a.finished + 3 * interval '1 hour'),
	   case when doctor.profession_name is null
				then 'Дежурный врач'
				else doctor.profession_name
		end doctor_profession
from appointment a
left join patient p on a.patient_id = p.id 
left join person per on p.person_id = per.id 
left join promotion promo on a.promotion_id = promo.id
----
left join doctor on a.doctor_id = doctor.id
----
inner join product_condition pc
   on promo.product_condition_id = pc.id
----
where a.finished at time zone 'UTC' > '2020-03-30'
and a.finished < timezone('UTC', now() - (extract(hour from now())) * interval '1 hour' - (extract(minute from now())) * interval '1 minute' - (extract(second from now())) * interval '1 second')
and a.good
	and not (
			a.patient_id in (
							select patient_id from patient_categories pcat
							inner join reference rf 
							on rf.id = pcat.reference_id
							where rf.code = 'TEST'
			)
			or lower(a.report_comment) = 'тест'
	)
)
select * from temp
where "name" = 'VEBMED'

	
	
	


"""

In [4]:
connection = UniversalConnection('../../../.credentials/telemed/prom.cfg', DBType.Postgres)

2020-05-29 17:45:25 connect to postgres database using config file "../../../.credentials/telemed/prom.cfg"
2020-05-29 17:45:25 creating ssh tunnel to 172.16.100.19 as root...
2020-05-29 17:45:25 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 44493
2020-05-29 17:45:25 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None


In [5]:
data = connection.query(sql)

2020-05-29 17:45:33 @telemed query:
                    with temp as (
				    select a.id appoint_id,
				    	   p.person_id,
				    	   per.phone,
				    	   --per.partner_id,
				    	   --par."name", 
				    	   case when pc.product_id in (
				                select id from product
				                where full_name like '%ВЭБ%') then 'VEBMED'
				            	else 'ПРОЧИЕ'
				            end "name",
				    	   DATE(a.finished + 3 * interval '1 hour'),
				    	   case when doctor.profession_name is null
				    				then 'Дежурный врач'
				    				else doctor.profession_name
				    		end doctor_profession
				    from appointment a
				    left join patient p on a.patient_id = p.id 
				    left join person per on p.person_id = per.id 
				    left join promotion promo on a.promotion_id = promo.id
				    ----
				    left join doctor on a.doctor_id = doctor.id
				    ----
				    inner join product_condition pc
				       on promo.product_condition_id = pc.id
				    --

In [6]:
connection.close()

2020-05-29 17:45:47 @telemed: closing postgres connection...
2020-05-29 17:45:47 closing ssh tunnel to 172.16.100.19...


In [8]:
df = pd.DataFrame(data).copy()

In [9]:
df.head()

,appoint_id,person_id,phone,name,date,doctor_profession
0,127919,2546091,79518565014,VEBMED,2020-03-31,Врач общей практики (Семейный врач)
1,144566,3521114,79687101977,VEBMED,2020-04-15,Уролог
2,135952,3201364,79911116364,VEBMED,2020-04-07,Дежурный врач
3,173510,3557449,79605280008,VEBMED,2020-04-30,Дежурный врач
4,152628,3531222,79536239692,VEBMED,2020-04-21,Эндокринолог


In [10]:
df['dirty_region'] = df['phone'].apply(get_region_by_two_bases)

In [11]:
df['region'] = df['dirty_region'].apply(clean_region)

In [12]:
df.head(15)

,appoint_id,person_id,phone,name,date,doctor_profession,dirty_region,region
0,127919,2546091,79518565014,VEBMED,2020-03-31,Врач общей практики (Семейный врач),Воронежская обл.,Воронежская область
1,144566,3521114,79687101977,VEBMED,2020-04-15,Уролог,г. Москва и Московская область,Москва и Московская область
2,135952,3201364,79911116364,VEBMED,2020-04-07,Дежурный врач,Ставропольский край,Ставропольский край
3,173510,3557449,79605280008,VEBMED,2020-04-30,Дежурный врач,Ярославская обл.,Ярославская область
4,152628,3531222,79536239692,VEBMED,2020-04-21,Эндокринолог,Орловская обл.,Орловская область
5,208128,3616025,79953064562,VEBMED,2020-05-17,Дежурный врач,Пензенская обл.,Пензенская область
6,131310,3084235,79166063464,VEBMED,2020-04-04,Аллерголог-иммунолог,г. Москва и Московская область,Москва и Московская область
7,153655,3540638,79996980789,VEBMED,2020-04-21,Дежурный врач,Ростовская обл.,Ростовская область
8,146190,3229586,79107992557,VEBMED,2020-04-16,Эндокринолог,Нижегородская обл.,Нижегородская область
9,168680,3563887,79959304082,VEBMED,2020-04-28,Дежурный врач,Ростовская обл.,Ростовская область


In [15]:
df.groupby(['region', 'doctor_profession'])[['appoint_id']].count().reset_index().rename(columns={'appoint_id':'consultations'}).to_excel('doctors-regions.xlsx', index=False)